In [ ]:
#Q1 - SQL

In [ ]:
SELECT
    actor_id,
    director_id
FROM
    ActorDirector
GROUP BY
    actor_id, director_id
HAVING
    COUNT(*) >= 3;


In [1]:
#pandas
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    # Group by actor_id and director_id, count occurrences
    result = (
        actor_director
        .groupby(['actor_id', 'director_id'])
        .size()
        .reset_index(name='count')
    )

    # Filter pairs with at least 3 collaborations
    result = result[result['count'] >= 3][['actor_id', 'director_id']]

    return result


In [ ]:
#Q2

In [ ]:
#SQL
SELECT
    user_id,
    CONCAT(UPPER(SUBSTRING(name, 1, 1)), LOWER(SUBSTRING(name, 2))) AS name
FROM
    Users
ORDER BY
    user_id;


In [ ]:
#pandas

import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    # Capitalize the first letter of each name and make the rest lowercase
    users['name'] = users['name'].str.capitalize()

    # Sort by user_id
    users = users.sort_values('user_id').reset_index(drop=True)

    return users

In [ ]:
#Q3

In [ ]:
#SQL

SELECT
    p.firstName,
    p.lastName,
    a.city,
    a.state
FROM
    Person p
LEFT JOIN
    Address a
ON
    p.personId = a.personId;


In [ ]:
#pandas

import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    # Perform a left join on personId
    result = pd.merge(person, address, how='left', left_on='personId', right_on='personId')

    # Select the required columns
    result = result[['firstName', 'lastName', 'city', 'state']]

    return result

In [ ]:
#Q4

#sql

SELECT
    (SELECT DISTINCT salary
     FROM Employee
     ORDER BY salary DESC
     LIMIT 1 OFFSET 1) AS SecondHighestSalary;


In [ ]:
#pandas

import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    # Get unique salaries and sort them descending
    unique_salaries = employee['salary'].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)

    # Get the second highest salary if it exists, else None
    second_salary = unique_salaries[1] if len(unique_salaries) > 1 else None

    # Return as a DataFrame
    return pd.DataFrame({'SecondHighestSalary': [second_salary]})


In [ ]:
#Q5

#SQL

SELECT
    p.product_name,
    SUM(o.unit) AS unit
FROM
    Orders o
JOIN
    Products p
ON
    o.product_id = p.product_id
WHERE
    o.order_date >= '2020-02-01'
    AND o.order_date < '2020-03-01'
GROUP BY
    p.product_id, p.product_name
HAVING
    SUM(o.unit) >= 100;


In [ ]:
#pandas

import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    # Filter orders for February 2020
    feb_orders = orders[(orders['order_date'] >= '2020-02-01') & (orders['order_date'] < '2020-03-01')]

    # Group by product_id and sum the units
    total_units = feb_orders.groupby('product_id', as_index=False)['unit'].sum()

    # Keep only products with at least 100 units
    total_units = total_units[total_units['unit'] >= 100]

    # Merge with products table to get product_name
    result = pd.merge(total_units, products[['product_id', 'product_name']], on='product_id')

    # Select required columns
    result = result[['product_name', 'unit']]

    return result


In [2]:
#Q6

#sql

SELECT
    u.unique_id,
    e.name
FROM
    Employees e
LEFT JOIN
    EmployeeUNI u
ON
    e.id = u.id;


In [ ]:
#pandas

import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    # Perform a left join on 'id'
    result = pd.merge(employees, employee_uni[['id', 'unique_id']], on='id', how='left')

    # Select the required columns
    result = result[['unique_id', 'name']]

    return result


In [2]:
#Q7
#SQL
WITH first_login AS (
    SELECT
        player_id,
        MIN(event_date) AS first_day
    FROM Activity
    GROUP BY player_id
),
next_day_login AS (
    SELECT
        f.player_id
    FROM first_login f
    JOIN Activity a
      ON f.player_id = a.player_id
     AND a.event_date = DATE_ADD(f.first_day, INTERVAL 1 DAY)
)
SELECT
    ROUND(
        COUNT(DISTINCT n.player_id) / COUNT(DISTINCT f.player_id), 2
    ) AS fraction
FROM first_login f
LEFT JOIN next_day_login n
    ON f.player_id = n.player_id;


In [ ]:
#pandas

import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    # Convert event_date to datetime if not already
    activity['event_date'] = pd.to_datetime(activity['event_date'])

    # Find first login date per player
    first_login = activity.groupby('player_id')['event_date'].min().reset_index()
    first_login.rename(columns={'event_date': 'first_day'}, inplace=True)

    # Merge with original activity to see if player logged in the next day
    merged = pd.merge(activity, first_login, on='player_id', how='left')

    # Check if player logged in the day after their first login
    merged['next_day_login'] = merged['event_date'] == (merged['first_day'] + pd.Timedelta(days=1))

    # Count fraction
    fraction = merged.groupby('player_id')['next_day_login'].any().mean()

    # Return as DataFrame rounded to 2 decimal places
    return pd.DataFrame({'fraction': [round(fraction, 2)]})


In [ ]:
#Q8
#SQL

SELECT
    p.project_id,
    ROUND(AVG(e.experience_years), 2) AS average_years
FROM
    Project p
JOIN
    Employee e
ON
    p.employee_id = e.employee_id
GROUP BY
    p.project_id;


In [ ]:
#pandas

import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    # Merge project with employee to get experience_years
    merged = pd.merge(project, employee[['employee_id', 'experience_years']], on='employee_id', how='left')

    # Group by project_id and calculate average experience, rounded to 2 decimals
    result = merged.groupby('project_id', as_index=False)['experience_years'].mean()
    result.rename(columns={'experience_years': 'average_years'}, inplace=True)
    result['average_years'] = result['average_years'].round(2)

    return result


In [ ]:
#Q9
#SQL

WITH ranked AS (
    SELECT
        e.name AS Employee,
        e.salary AS Salary,
        d.name AS Department,
        DENSE_RANK() OVER (PARTITION BY e.departmentId ORDER BY e.salary DESC) AS rnk
    FROM Employee e
    JOIN Department d
      ON e.departmentId = d.id
)
SELECT
    Department,
    Employee,
    Salary
FROM ranked
WHERE rnk <= 3
ORDER BY Department, Salary DESC;


In [ ]:
#pandas
import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    # Merge employee with department to get department names
    merged = pd.merge(employee, department, left_on='departmentId', right_on='id', how='left')

    # Sort by department and descending salary
    merged.sort_values(['departmentId', 'salary'], ascending=[True, False], inplace=True)

    # Assign rank per department based on salary using dense rank
    merged['rank'] = merged.groupby('departmentId')['salary'].rank(method='dense', ascending=False)

    # Keep only top 3 ranks
    top_employees = merged[merged['rank'] <= 3]

    # Select required columns
    result = top_employees[['name_y', 'name_x', 'salary']].copy()
    result.columns = ['Department', 'Employee', 'Salary']

    # Optional: sort by Department and Salary descending
    result.sort_values(['Department', 'Salary'], ascending=[True, False], inplace=True)

    return result.reset_index(drop=True)
